The data contains a pair of paragraphs. These text paragraphs are randomly sampled from a raw dataset. Each pair of the sentence may or may not be semantically similar. The dataset considered for this project does not contain any labels. Given below is the solution to the unsupervised machine learning problem

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
import re
import scipy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec, KeyedVectors, FastText
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Flatten, SimpleRNN, RNN,GRU, SpatialDropout1D, Dropout
from keras.preprocessing.sequence import pad_sequences


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('../input/Text_Similarity_Dataset.csv')
data.shape #(4023,3)

X = data.iloc[:,1:].values
s1 = X[:,0]
s2 = X[:,1]


In [ ]:
s1 = ['What is the step by step guide to invest in share market in india?',
'What is the story of Kohinoor (Koh-i-Noor) Diamond?',
'How can I increase the speed of my internet connection while using a VPN?',
'Why am I mentally very lonely? How can I solve it?',
'Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?',
'Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?',
'Should I buy tiago?',
'How can I be a good geologist?',
'When do you use ã‚· instead of ã—?',
'Motorola (company): Can I hack my Charter Motorolla DCX3400?',
'Method to find separation of slits using fresnel biprism?',
'How do I read and find my YouTube comments?',
'What can make Physics easy to learn?',
'What was your first sexual experience like?',
'What are the laws to change your status from a student visa to a green card in the US, how do they compare to the immigration laws in Canada?',
'What would a Trump presidency mean for current international masterâ€™s students on an F1 visa?']

s2 = ['What is the step by step guide to invest in share market?',
'What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?',
'How can Internet speed be increased by hacking through DNS?',
'Find the remainder when [math]23^{24}[/math] is divided by 24,23?',
'Which fish would survive in salt water?',
"I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?",
'What keeps childern active and far from phone and video games?',
'What should I do to be a great geologist?',
'When do you use "&" instead of "and"?',
'How do I hack Motorola DCX3400 for free internet?',
'What are some of the things technicians can tell about the durability and reliability of Laptops and its components?',
'How can I see all my Youtube comments?',
'How can you make physics easy to learn?',
'What was your first sexual experience?',
'What are the laws to change your status from a student visa to a green card in the US? How do they compare to the immigration laws in Japan?',
'How will a Trump presidency affect the students presently in US or planning to study in US?'
]

# Preprocessing of Data

In [ ]:
tokens1 = []
tokens2 = []
tokens1 = [word_tokenize(str(sentence)) for sentence in s1]
tokens2 = [word_tokenize(str(sentence)) for sentence in s2]

rm1 = []
for w in tokens1:
    sm = re.sub('[^A-Za-z]',' ', str(w))
    x = re.split("\s", sm)
    rm1.append(x)
    
rm2 = []
for w in tokens2:
    sm2 = re.sub('[^A-Za-z]',' ',str(w))
    x2 = re.split("/s",sm2)
    rm2.append(x2)


In [ ]:
#Removing whitespaces    
for sent in rm1:
    while '' in sent:
        sent.remove('')

for sent in rm2:
    while '' in sent:
        sent.remove('')
        
# Lowercasing
low1 = []
for i in rm1:
    i = [x.lower() for x in i]
    low1.append(i)

low2 = []
for i in rm2:
    i = [x.lower() for x in i]
    low2.append(i)


In [ ]:
#Lemmatozation
lemma1 = []
wnl = WordNetLemmatizer()
for sent in low1:
    tokens = [wnl.lemmatize(w) for w in sent]
    lemma1.append(tokens)
    
lemma2 = []
for sent in low2:
    tok = [wnl.lemmatize(se) for se in sent]
    lemma2.append(tok)
    
# Removing Stopwords
filter_words1 = []
Stopwords = set(stopwords.words('english'))

for sent in lemma1:
    tokens = [w for w in sent if w not in Stopwords]
    filter_words1.append(tokens)
    
filter_words2 = []
for sent in lemma2:
    tokens2 = [w for w in sent if w not in Stopwords]
    filter_words2.append(tokens2)


# FastText
FastText is an extension to Word2Vec proposed by Facebook in 2016. Instead of feeding individual words into the Neural Network, FastText breaks words into several n-grams (sub-words). For instance, the tri-grams for the word apple is app, ppl, and ple (ignoring the starting and ending of boundaries of words). The word embedding vector for apple will be the sum of all these n-grams. After training the Neural Network, we will have word embeddings for all the n-grams given the training dataset. Rare words can now be properly represented since it is highly likely that some of their n-grams also appears in other words.


In [ ]:
model = FastText('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz')
word_vectors = model.wv
vocabulary = word_vectors.vocab.items()
similarity = word_vectors.similarity('woman', 'man')

def cosine_similarity_bw_two_words(word1,word2):
    return word_vectors.similarity(word1,word2)


## Cosine Similarity
Cosine similarity is a measure of similarity between two non-zero vectors of an inner product space that measures the cosine of the angle between them. The cosine of 0° is 1, and it is less than 1 for any angle in the interval (0, π] radians.


In [ ]:
result = []
wmd = []
for sent1, sent2 in zip(filter_words1,filter_words2):
    print(sent1)
    print(sent2)
    vector1 = np.mean([model[word] for word in sent1], axis = 0)
    vector2 = np.mean([model[word] for word in sent2], axis = 0)
    cosine = scipy.spatial.distance.cosine(vector1, vector2)
    result.append((1-cosine))
    wmd.append(model.wmdistance(sent1,sent2))
#data['Result[%]'] = result

In [ ]:
result

In [ ]:
wmd

In [ ]:
data

In [ ]:
data.sort_values(["Result[%]"], axis=0, 
                 ascending=False) 